In [1]:
import os
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [2]:
TEST_DATASET_PATH = '../../PR/test_dataset'
TEST_FILES = os.listdir(TEST_DATASET_PATH)
print(f'[verify] Found {len(TEST_FILES)} Test Files\n')

'''
all_labels_test = []

for file in test_files_pb:
    file_path = os.path.join(TEST_DATASET_PATH, file)
    data = pd.read_csv(file_path)
    all_labels_test.extend(data['type'].unique())

label_encoder = LabelEncoder()
label_encoder.fit(all_labels_test)
'''

X_test_all = []

test_files_pb = tqdm(TEST_FILES,'[preproc.transform] Transforming Data...')

for file in test_files_pb:
    file_path = os.path.join(TEST_DATASET_PATH, file)
    data = pd.read_csv(file_path)

    # 转换时间列，提取特征等
    data['time'] = pd.to_datetime(data['time'])
    data['hour'] = data['time'].dt.hour
    data['day_of_week'] = data['time'].dt.dayofweek
    data['month'] = data['time'].dt.month

    X_test = data[['渔船ID', 'lat', 'lon', '速度', '方向', 'hour', 'day_of_week', 'month']]
    #X_test = data[['lat', 'lon', '速度', '方向', 'hour', 'day_of_week', 'month']]

    X_test_all.append(X_test)

# 将所有数据合并为一个大的 DataFrame
X_test = pd.concat(X_test_all, ignore_index=True)
X_test

[verify] Found 4034 Test Files



[preproc.transform] Transforming Data...: 100%|██████████| 4034/4034 [00:52<00:00, 76.59it/s] 


,渔船ID,lat,lon,速度,方向,hour,day_of_week,month
0,18330,30.685278,122.232972,8.0,22.0,7,5,10
1,18330,30.694222,122.241833,8.6,40.0,7,5,10
2,18330,30.702194,122.252056,8.6,44.0,7,5,10
3,18330,30.719944,122.269750,8.9,40.0,7,5,10
4,18330,30.728972,122.278306,9.4,42.0,7,5,10
...,...,...,...,...,...,...,...,...
6422980,22363,30.940000,123.557889,3.4,152.0,3,6,10
6422981,22363,30.935917,123.560111,3.2,156.0,3,6,10
6422982,22363,30.921333,123.573750,3.3,135.0,3,6,10
6422983,22363,30.918056,123.577806,3.6,130.0,3,6,10


In [ ]:
from joblib import load
rf_classifier = load('../../PR/RandomForestClassifier.joblib')

In [ ]:
y_test_pred = rf_classifier.predict(X_test[['lat', 'lon', '速度', '方向', 'hour', 'day_of_week', 'month']])
y_test_pred

In [ ]:
X_test['type'] = y_test_pred
#X_test.loc[X_test['prediction' == 0], 'prediction'] = '刺网'
X_duplicates = X_test[['渔船ID', 'type']].replace({0:"刺网", 1:"围网", 2:"拖网"})

In [ ]:
X_final = X_duplicates.drop_duplicates(subset='渔船ID', keep='first')
X_final

In [ ]:
X_final.to_csv('../../PR/submissions/rfc/submission_rfc.csv', index=False)